In [1]:
import cmipaccess as cmip
import numpy as np

In [3]:
cmip.spiritx.load_toa_energy_budget('CESM2','amip-p4K', no_parent=True)

  0%|          | 0/4 [00:00<?, ?it/s]/home/rguillaume/virtual_envs/phd/lib/python3.10/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(
100%|██████████| 4/4 [00:02<00:00,  1.56it/s]


<xarray.Dataset> Size: 20kB
Dimensions:      (time: 420, realisation: 1)
Coordinates:
  * time         (time) object 3kB 1979-01-15 12:00:00 ... 2013-12-15 12:00:00
  * realisation  (realisation) <U8 32B 'r1i1p1f1'
Data variables:
    tas          (realisation, time) float64 3kB 288.9 289.7 ... 291.1 290.3
    rlut         (realisation, time) float64 3kB 241.4 245.8 ... 245.8 245.4
    rsdt         (realisation, time) float64 3kB 351.6 348.9 ... 347.7 351.2
    rsut         (realisation, time) float64 3kB 100.5 98.49 ... 100.4 105.7
    eei          (realisation, time) float64 3kB 9.672 4.614 ... 1.476 0.0391

In [2]:
cmip.get_CMIP6_data('CESM2','amip-p4K', 'r1i1p1f1', 'rsdt', chunks = dict(time=10))

/home/rguillaume/virtual_envs/phd/lib/python3.10/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.16.0
  warnings.warn(
/home/rguillaume/virtual_envs/phd/lib/python3.10/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'rsdt' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)


<xarray.Dataset> Size: 93MB
Dimensions:    (lat: 192, lon: 288, time: 420, nbnd: 2)
Coordinates:
  * lat        (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * time       (time) object 3kB 1979-01-15 12:00:00 ... 2013-12-15 12:00:00
Dimensions without coordinates: nbnd
Data variables:
    time_bnds  (time, nbnd) object 7kB dask.array<chunksize=(10, 2), meta=np.ndarray>
    lat_bnds   (lat, nbnd) float32 2kB dask.array<chunksize=(192, 2), meta=np.ndarray>
    lon_bnds   (lon, nbnd) float32 2kB dask.array<chunksize=(288, 2), meta=np.ndarray>
    rsdt       (time, lat, lon) float32 93MB dask.array<chunksize=(10, 192, 288), meta=np.ndarray>
Attributes: (12/46)
    Conventions:                     CF-1.7 CMIP-6.2
    activity_id:                     CFMIP
    branch_method:                   no parent
    branch_time_in_child:            721970.0
    branch_time_in_parent:           0.0
    case_id:                         61
    ...                              ...
    table_id:                        Amon
    tracking_id:                     hdl:21.14100/0602d1bb-737f-436d-8403-df9...
    variable_id:                     rsdt
    variant_info:                    CFMIP AMIP-p4k historical with active la...
    variant_label:                   r1i1p1f1
    DODS_EXTRA.Unlimited_Dimension:  time

In [4]:
cmip.esgf.find_models_experiment('hist-volc')

['CMCC-CM2-SR5', 'CanESM5', 'GISS-E2-1-G', 'MIROC6', 'MPI-ESM1-2-LR']

In [7]:
cmip.get_path_CMIP6_data_esgf('CanESM5', 'piClim-histall','r1i1p2f1', 'tas')

['http://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/RFMIP/CCCma/CanESM5/piClim-histall/r1i1p2f1/Amon/tas/gn/v20190429/tas_Amon_CanESM5_piClim-histall_r1i1p2f1_gn_185001-201412.nc',
 'http://esgf-data1.llnl.gov/thredds/dodsC/css03_data/CMIP6/RFMIP/CCCma/CanESM5/piClim-histall/r1i1p2f1/Amon/tas/gn/v20190429/tas_Amon_CanESM5_piClim-histall_r1i1p2f1_gn_201501-210012.nc']

In [3]:
from pyesgf.search import SearchConnection
conn = SearchConnection('https://esgf-data.dkrz.de/esg-search', distrib=True)

In [33]:
ctx = conn.new_context(
    project='CMIP5',
    experiment='piControl',
    model='CCSM4',
    # ensemble='r1i1p1',
    time_frequency='mon',
    facets='model',
    data_node='esgf2.dkrz.de',
    )
ctx.hit_count
result = ctx.search()[0]
variable='tos'
# result = [k for k in ctx.search() if f".{variable}." in k ]
result.dataset_id
files = result.file_context().search()
for file in files:
    if 'tasmax' in file.opendap_url:
        tasmax_url = file.opendap_url
        print(tasmax_url)


-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


TypeError: argument of type 'NoneType' is not iterable

In [53]:
files = ctx.search()[1].file_context().search()


-------------------------------------------------------------------------------
Warning - defaulting to search with facets=*

This behavior is kept for backward-compatibility, but ESGF indexes might not
successfully perform a distributed search when this option is used, so some
results may be missing.  For full results, it is recommended to pass a list of
facets of interest when instantiating a context object.  For example,

      ctx = conn.new_context(facets='project,experiment_id')

Only the facets that you specify will be present in the facets_counts dictionary.

This warning is displayed when a distributed search is performed while using the
facets=* default, a maximum of once per context object.  To suppress this warning,
set the environment variable ESGF_PYCLIENT_NO_FACETS_STAR_WARNING to any value
or explicitly use  conn.new_context(facets='*')

-------------------------------------------------------------------------------


In [54]:
for file in files:
    print(file.opendap_url)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
